In [2]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np

# corregir puntos de stops

In [3]:
def splitPoint(row):
    row['latitude'] = row['WKT'].split()[2].strip()[:-1]
    row['longitude'] = row['WKT'].split()[1].strip()[1:]
    return row

data_busstops = pd.read_csv('stop_points.csv')
# data_busstops = data_busstops.apply(splitPoint, axis=1)
data_busstops = data_busstops.sort_values(by='number').reset_index(drop=True)
data_busstops = data_busstops.drop(['number', 'WKT'], axis=1) # escoger los paraderos de ida
# data_busstops['id'] = range(36, len(data_busstops)+36)
data_busstops['id'] = data_busstops['id'].apply(lambda id: str(int(id)))
data_busstops['navigation'] = data_busstops['navigation'].values.astype(np.int64)
data_busstops['radio'] = data_busstops['radio'].values.astype(np.int64)

# data_busstops.append(pd.Series([0, 'start', 0, ]))
data_busstops

KeyError: 'number'

In [39]:
# df_stops_ida = pd.read_csv('busstops_ida.csv')
# data_busstops['radio'] = df_stops_ida['radio']
# data_busstops

In [10]:
data_busstops.to_csv('busstops_ida.csv')

# Extraer en CSV

In [32]:
# leer dataset monitoring
dt = pd.read_csv('_Process_monitoring__202303251137.csv')
dt['vehicle_id_id'].unique()


# diseño del df: creacion data_time y latitude_longitude
dt['date_time'] = dt['date']  + ' ' + dt['time']
dt['latitude'] = dt.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
dt.drop(['longitude'], inplace=True, axis=1)
dt = dt.rename(columns={'latitude':'latitude_longitude', 'vehicle_id_id':'vehicle_id'}, inplace=False)
dt = dt.drop_duplicates(subset=['latitude_longitude']) # (65155, 7) (64702, 7)
dt = dt.sort_values(by='date_time')

# convertir de object a datatime
dt['date_time'] = pd.to_datetime(dt['date_time'], format='%Y-%m-%d %H:%M:%S')
print(dt.shape)
dt

(1671363, 9)


,id,date,time,lap,latitude_longitude,velocity,navigation,vehicle_id,date_time
29756,13172252,2023-03-01,00:01:34,5,"(-13.5513563, -71.9902231)",1,140,SJ40,2023-03-01 00:01:34
29311,13172234,2023-03-01,00:01:49,0,"(-13.5506013, -71.8786173)",2,185,SJ10,2023-03-01 00:01:49
29780,13172253,2023-03-01,00:03:25,0,"(-13.5506231, -71.8786667)",3,206,SJ10,2023-03-01 00:03:25
31234,13172309,2023-03-01,00:05:13,5,"(-13.5383472, -71.9807985)",0,0,SJ08,2023-03-01 00:05:13
29831,13172255,2023-03-01,00:05:30,0,"(-13.55065, -71.8785454)",3,113,SJ10,2023-03-01 00:05:30
...,...,...,...,...,...,...,...,...,...
1585989,17270532,2023-03-25,11:37:16,2,"(-13.5504177, -71.9870624)",31,230,SJ03,2023-03-25 11:37:16
1585942,17270531,2023-03-25,11:37:16,3,"(-13.550582, -71.8741605)",16,285,SJ20,2023-03-25 11:37:16
1586202,17270537,2023-03-25,11:37:17,3,"(-13.5217018, -71.9646513)",19,289,SJ32,2023-03-25 11:37:17
1586071,17270534,2023-03-25,11:37:17,2,"(-13.5405389, -71.9812735)",38,13,SJ04,2023-03-25 11:37:17


In [33]:
# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    dnav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (dnav<=45 and dis<=stop['radio'])

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if ( itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

In [34]:
dt['vehicle_id'].unique()

array(['SJ40', 'SJ10', 'SJ08', 'SJ37', 'SJ27', 'SJ28', 'SJ03', 'SJ05',
       'SJ14', 'SJ31', 'SJ20', 'SJ16', 'SJ41', 'SJ35', 'SJ34', 'SJ39',
       'SJ04', 'SJ01', 'SJ06', 'SJ42', 'SJ36', 'SJ23', 'SJ24', 'SJ33',
       'SJ38', 'SJ29', 'SJ21', 'SJ32', 'SJ25', 'SJ30', 'SJ09', 'SJ07',
       'SJ26', 'SJ22', 'SJ15', 'SJ13', 'SJ12', 'SJ18', 'SJ17', 'SJ43',
       'SJ19', 'SJ11', 'SJ02'], dtype=object)

In [59]:
def quierodatos(veh, fecha):
    # parametros de extraccion
    # veh = 'SJ23'
    # fecha = '2023-03-20'
    dt_veh = dt[(dt['vehicle_id'] == veh) & (dt['date'] == fecha)]
    laps = sorted(list(dt_veh['lap'].unique())[1:-1])
    # laps = [2] 
    if len(laps)== 0:
        print(f'No hay nada {veh}-{fecha}')
        return False
     
    print(sorted(list(dt_veh['lap'].unique())), '->', laps)

    dt_vehlap = pd.DataFrame()
    for lap in laps:
        tmp =  idaPoints(dt_veh[dt_veh['lap']==lap].reset_index(drop=True))
        dt_vehlap = pd.concat([dt_vehlap, tmp])
        print(f"{veh} {lap} -> {tmp.shape[0]}")

    # print(dt_vehlap)
    if len(dt_vehlap['lap'].unique()) != 1:
        name = 'monitoring_veh/' + str(veh) + '-' + fecha + '.csv'
        dt_vehlap.to_csv(name)
        print(f'print data ({veh}, {laps}, {dt_vehlap.shape[0]}) -> {name}')
    else:
        name = 'monitoring_veh_lap/' + str(veh) + '-lap' + str(dt_vehlap['lap'].unique()[0]) + '.csv'
        dt_vehlap.to_csv(name)
        print(f'print data ({veh}, {laps}, {dt_vehlap.shape[0]}) -> {name}')

    return dt_vehlap

In [63]:
quierodatos('SJ30', '2023-03-20')
quierodatos('SJ39', '2023-03-17')
quierodatos('SJ01', '2023-03-11')
quierodatos('SJ28', '2023-03-01')
quierodatos('SJ22', '2023-03-25')

[0, 1, 2, 3, 4, 5] -> [1, 2, 3, 4]
SJ30 1 -> 204
SJ30 2 -> 186
SJ30 3 -> 11
SJ30 4 -> 212
print data (SJ30, [1, 2, 3, 4], 613) -> monitoring_veh/SJ30-2023-03-20.csv
[0, 1, 2, 3, 4, 5] -> [1, 2, 3, 4]
SJ39 1 -> 190
SJ39 2 -> 182
SJ39 3 -> 192
SJ39 4 -> 198
print data (SJ39, [1, 2, 3, 4], 762) -> monitoring_veh/SJ39-2023-03-17.csv
[0, 1, 2, 3, 4, 5] -> [1, 2, 3, 4]
SJ01 1 -> 70
SJ01 2 -> 141
SJ01 3 -> 211
SJ01 4 -> 217
print data (SJ01, [1, 2, 3, 4], 639) -> monitoring_veh/SJ01-2023-03-11.csv
[0, 1, 2, 3, 4, 5] -> [1, 2, 3, 4]
SJ28 1 -> 209
SJ28 2 -> 201
SJ28 3 -> 201
SJ28 4 -> 207
print data (SJ28, [1, 2, 3, 4], 818) -> monitoring_veh/SJ28-2023-03-01.csv
[0, 1, 2, 3] -> [1, 2]
SJ22 1 -> 198
SJ22 2 -> 202
print data (SJ22, [1, 2], 400) -> monitoring_veh/SJ22-2023-03-25.csv


,id,date,time,lap,latitude_longitude,velocity,navigation,vehicle_id,date_time
0,17211445,2023-03-25,05:24:07,1,"(-13.5555831, -71.8562602)",11,301,SJ22,2023-03-25 05:24:07
1,17211464,2023-03-25,05:24:22,1,"(-13.555159, -71.8568393)",20,303,SJ22,2023-03-25 05:24:22
2,17211484,2023-03-25,05:24:39,1,"(-13.5547375, -71.8574292)",16,302,SJ22,2023-03-25 05:24:39
3,17211515,2023-03-25,05:25:04,1,"(-13.5543432, -71.8580665)",16,302,SJ22,2023-03-25 05:25:04
4,17211530,2023-03-25,05:25:16,1,"(-13.5539887, -71.8587541)",30,294,SJ22,2023-03-25 05:25:16
...,...,...,...,...,...,...,...,...,...
197,17248511,2023-03-25,09:29:44,2,"(-13.5518263, -71.9911424)",22,235,SJ22,2023-03-25 09:29:44
198,17248549,2023-03-25,09:29:56,2,"(-13.5521097, -71.9908819)",22,66,SJ22,2023-03-25 09:29:56
199,17248574,2023-03-25,09:30:08,2,"(-13.5515767, -71.9900494)",36,53,SJ22,2023-03-25 09:30:08
200,17248610,2023-03-25,09:30:20,2,"(-13.5512762, -71.9889825)",37,121,SJ22,2023-03-25 09:30:20
